# WeatherPy

---

## Starter Code to Generate Random Geographic Coordinates and a List of Cities

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import the OpenWeatherMap API key
from api_keys import weather_api_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

### Generate the Cities List by Using the `citipy` Library

In [ ]:
# Empty list for holding the latitude and longitude combinations
lat_lngs = []

# Empty list for holding the cities names
cities = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name

    # If the city is unique, then add it to our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(f"Number of cities in the list: {len(cities)}")

---

## Requirement 1: Create Plots to Showcase the Relationship Between Weather Variables and Latitude

### Use the OpenWeatherMap API to retrieve weather data from the cities list generated in the started code

In [ ]:
# Set the API base URL
url = "http://api.openweathermap.org/data/2.5/weather?"

# Define an empty list to fetch the weather data for each city
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    # Create endpoint URL with each city
    city_url = city_url = url + "&q=" + city
    
    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()

        # Parse out latitude, longitude, max temp, humidity, cloudiness, wind speed, country, and date
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_date = city_weather["dt"]

        # Append the City information into city_data list
        city_data.append({"City": city, 
                          "Lat": city_lat, 
                          "Lng": city_lng, 
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
              
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [ ]:
# Convert the cities weather data into a Pandas DataFrame
city_data_df = pd.DataFrame(city_data)

In [ ]:
# Show Record Count
city_data_df.count()

In [ ]:
# Display sample data
city_data_df.head()

In [ ]:
# Export the City_Data into a csv
city_data_df.to_csv("output_data/cities.csv", index_label="City_ID")

In [ ]:
# Read saved data
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

### Create the Scatter Plots Requested

#### Latitude Vs. Temperature

In [ ]:
plt.scatter(     
    city_data_df["Lat"],
    city_data_df["Max Temp"],
    edgecolor="black",
    linewidths=1,
    marker="o", 
    alpha=0.8,
    label="Cities"
)

#### Latitude Vs. Humidity

In [ ]:
# Build the scatter plots for latitude vs. humidity
plt.scatter(city_data_df["Lat"],city_data_df["Cloudiness"], edgecolors="black",
label="Cities"
)

# Incorporate the other graph properties
plt.title("City Latitude vs. Humidity (%s)" % time.strftime("%Y-%m-%d"))
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.grid(True)

# Save the figure
plt.savefig("output_data/Fig2.png")

# Show plot
plt.show()

#### Latitude Vs. Cloudiness

In [ ]:
# Build the scatter plots for latitude vs. cloudiness
plt.scatter(city_data_df["Lat"],city_data_df["Cloudiness"], edgecolors="black",
label="Cities"
)

# Incorporate the other graph properties
plt.title("City Max Latitude vs. Cloudiness (2022-10-18)")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.grid(True)

# Save the figure
plt.savefig("output_data/Fig3.png")

# Show plot
plt.show()

#### Latitude vs. Wind Speed Plot

In [ ]:
# Build the scatter plots for latitude vs. wind speed
plt.scatter(city_data_df["Lat"],city_data_df["Wind Speed"], edgecolors="black",
label="Cities"
)

# Incorporate the other graph properties
plt.title("City Max Latitude vs. Wind Speed (2022-10-18)")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (m/s)")
plt.grid(True)

# Save the figure
plt.savefig("output_data/Fig4.png")

# Show plot
plt.show()

---

## Requirement 2: Compute Linear Regression for Each Relationship


In [ ]:
# Define a function to create Linear Regression plots
def plot_linear_regression(x_values, y_values, title, text_coordinates):
    
    # Compute linear regression
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
    regress_values = x_values * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

    # Linear Regression Plot
    plt.scatter(x_values,y_values)
    plt.plot(x_values,regress_values,"r-")
    plt.annotate(line_eq,text_coordinates,fontsize=15,color="red")
    plt.xlabel("Latitude")
    plt.ylabel(title)
    print(f"The r-value is: {rvalue**2}")
    plt.show()
    

In [ ]:
# Create a DataFrame with the Northern Hemisphere data (Latitude >= 0)
northern_hemi_df = city_data_df.loc[(city_data_df["Lat"] >= 0)]

# Display sample data
northern_hemi_df.head()

In [ ]:
# Create a DataFrame with the Southern Hemisphere data (Latitude < 0)
southern_hemi_df = city_data_df.loc[(city_data_df["Lat"] < 0)]

# Display sample data
southern_hemi_df.head()

###  Temperature vs. Latitude Linear Regression Plot

In [ ]:
# Linear regression on Northern Hemisphere
title = f"Northern Hemisphere City Latitude vs. Max Temp"
x_label = "Latitude"
y_label = "Max Temp (C)"
linearRegression(northern_hemi_df["Lat"], northern_hemi_df["Max Temp"], 0.05, 0.2, title, x_label, y_label)

In [ ]:
# Linear regression on Southern Hemisphere
title = f"Southern Hemisphere City Latitude vs. Max Temp"
x_label = "Latitude"
y_label = "Max Temp (C)"
linearRegression(southern_hemi_df["Lat"], southern_hemi_df["Max Temp"], 0.5, 0.1, title, x_label, y_label)

In [ ]:
**Discussion about the linear relationship**:The Northern Hemisphere model of random cities shows a strong negative linear path as the latitude moves toward higher latitudes the temperature decreases. This is confirmed by the r value which is 0.6856204686675623. The model for random cities in the Southern Hemisphere shows a somewhat strong correlation between latitude and temperature. As the data points move closer to the equator a significant number of the data points follow a positive linear path and the r value of 0.3061173519679404 which is less than .5 indicates the model is not as strong as the Northern Hemisphere model.

### Humidity vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
title = f"Northern Hemisphere City Latitude vs. Humidity"
x_label = "Latitude"
y_label = "Humidity"
linearRegression(northern_hemi_df["Lat"], northern_hemi_df["Humidity"], 0.5, 0.1, title, x_label, y_label)

In [ ]:
# Southern Hemisphere
title = f"Southern Hemisphere City Latitude vs. Humidity"
x_label = "Latitude"
y_label = "Humidity"
linearRegression(southern_hemi_df["Lat"], southern_hemi_df["Humidity"], 0.5, 0.1, title, x_label, y_label)

In [ ]:
**Discussion about the linear relationship:** The Northern Hemisphere model of random cities shows a very little correlation between the latitude and the humidity. This is confirmed by the r value which is 0.21373187591916867. The Southern Hemisphere model of random cities shows a no relative correlation between the latitude and the temperature. This is confirmed by the r value which is 0.08267817821577965.

### Cloudiness vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
title = f"Northern Hemisphere City Latitude vs. Cloudiness"
x_label = "Latitude"
y_label = "Cloudiness"
linearRegression(northern_hemi_df["Lat"], northern_hemi_df["Cloudiness"], 0.5, 0.1, title, x_label, y_label)

In [ ]:
# Southern Hemisphere
title = f"Southern Hemisphere City Latitude vs. Cloudiness"
x_label = "Latitude"
y_label = "Cloudiness"
linearRegression(southern_hemi_df["Lat"], southern_hemi_df["Cloudiness"], 0.5, 0.1, title, x_label, y_label)

In [ ]:
**Discussion about the linear relationship:** The datapoints for random cities in both models does not show any linear pattern between the latitude and cludiness for either hemisphere with the r values being very low at 0.102626179987376 for the Northern Hemisphere and an r value of 0.16212597513045385 for the Southern Hemisphere.

### Wind Speed vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
title = f"Northern Hemisphere City Latitude vs. Wind Speed"
x_label = "Latitude"
y_label = "Wind Speed"
linearRegression(northern_hemi_df["Lat"], northern_hemi_df["Wind Speed"], 0.5, 0.1, title, x_label, y_label)

In [ ]:
# Southern Hemisphere
title = f"Southern Hemisphere City Latitude vs. Wind Speed"
x_label = "Latitude"
y_label = "Wind Speed"
linearRegression(southern_hemi_df["Lat"], southern_hemi_df["Wind Speed"], 0.5, 0.1, title, x_label, y_label)

In [ ]:
**Discussion about the linear relationship:** The datapoints for random cities in both models does not show any linear pattern between the latitude and wind speed for either hemisphere with the r values being very low at 0.0008986382317618379 for the Northern Hemisphere and an r value of 0.12449790641835774 for the Southern Hemisphere.